In [ ]:
import nltk
import tensorflow as tf
import gensim
from __future__ import print_function, division
from future.utils import iteritems
from builtins import range
import pandas as pd
import csv
import sys
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from gensim.models.keyedvectors import KeyedVectors

In [ ]:
# Load Train Data
train_data = pd.read_csv('Train_cleaned2.csv', error_bad_lines=False, delimiter= '\t')

# Load the test data
test_data = pd.read_csv('/Test_CLEAN2.csv', error_bad_lines=False, delimiter= '\t')


# Pre-trained Glove 

In [ ]:


class GloveVectorizer:
  def __init__(self):
    # load in pre-trained word vectors
    print('Loading word vectors...')
    word2vec = {}
    embedding = []
    idx2word = []
    #with open( '/home/parisa/programming/data/Pre_trained_data/glove/twitterGlove/glove.twitter.27B.200d.txt') as f:
    with open('/home/parisa/nlp_class_dl/data/large_files/glove.6B/glove.6B.300d.txt') as f:
      # is just a space-separated text file in the format:
      # word vec[0] vec[1] vec[2] ...
      for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec
        embedding.append(vec)
        idx2word.append(word)
    print('Found %s word vectors.' % len(word2vec))

    # save for later
    self.word2vec = word2vec
    self.embedding = np.array(embedding)
    self.word2idx = {v:k for k,v in enumerate(idx2word)}
    print(self.embedding.shape)
    self.V, self.D = self.embedding.shape

  def fit(self, data):
    pass

  def transform(self, data):
    X = np.zeros((len(data), self.D))
    n = 0
    emptycount = 0
    for sentence in data:
      tokens = sentence.lower().split()
      vecs = []
      for word in tokens:
        if word in self.word2vec:
          vec = self.word2vec[word]
          vecs.append(vec)
      if len(vecs) > 0:
        vecs = np.array(vecs)
        X[n] = vecs.mean(axis=0)
      else:
        emptycount += 1
      n += 1
    print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))
    return X

  def fit_transform(self, data):
    self.fit(data)
    return self.transform(data)




In [ ]:
vectorizer = GloveVectorizer()
Xtrain = vectorizer.fit_transform(Train_data['conversations'])

# Pre-trained Word2vec

In [ ]:


class Word2VecVectorizer:
  def __init__(self):
    print("Loading in word vectors...")
    self.word_vectors = KeyedVectors.load_word2vec_format(
      '/home/parisa/nlp_class_dl/data/large_files/GoogleNews-vectors-negative300.bin',
      binary=True
    )
    print("Finished loading in word vectors")

  def fit(self, data):
    pass

  def transform(self, data):
    # determine the dimensionality of vectors
    v = self.word_vectors.get_vector('king')
    self.D = v.shape[0]

    X = np.zeros((len(data), self.D))
    n = 0
    emptycount = 0
    for sentence in data:
      tokens = sentence.split()
      vecs = []
      m = 0
      for word in tokens:
        try:
          # throws KeyError if word not found
          vec = self.word_vectors.get_vector(word)
          vecs.append(vec)
          m += 1
        except KeyError:
          pass
      if len(vecs) > 0:
        vecs = np.array(vecs)
        X[n] = vecs.mean(axis=0)
      else:
        emptycount += 1
      n += 1
    print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))
    return X


  def fit_transform(self, data):
    self.fit(data)
    return self.transform(data)

In [ ]:
vectorizer = Word2VecVectorizer()
Xtrain = vectorizer.fit_transform(Train_data['conversations'])